# Setup

Get the Data & Install Requirements

In [ ]:
# Download Datasets
!wget https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_single_train.tsv
!wget https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test-labels/lcp_single_test.tsv
!wget https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/trial/lcp_single_trial.tsv

In [ ]:
# Install Dependencies
!pip install sentence-transformers
!pip install transformers
!pip install textstat

In [ ]:
# Download & Unzip GloVe_42B Uncased Embeddings
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip glove.42B.300d.zip

Import Libraries


In [5]:
import os
import sys
import csv
import copy
from tqdm import tqdm, trange

import math
import numpy as np
import pandas as pd

import transformers
from transformers import BertTokenizer, BertModel

from sentence_transformers import SentenceTransformer

import string
from collections import Counter
import re

import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import spacy 
import textstat
from textstat.textstat import textstatistics,legacy_round
import textstat.textstat as textstat

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MinMaxScaler

from scipy import stats

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Load Data

In [6]:
traindf = pd.read_csv("lcp_single_train.tsv", encoding='utf-8', delimiter='\t', quotechar='\t', keep_default_na=False)
testdf = pd.read_csv("lcp_single_test.tsv", encoding='utf-8', delimiter='\t', quotechar='\t', keep_default_na=False)
trialdf = pd.read_csv("lcp_single_trial.tsv", encoding='utf-8', delimiter='\t', quotechar='\t', keep_default_na=False)
print(traindf.shape)
print(trialdf.shape)
print(testdf.shape)

(7662, 5)
(421, 5)
(917, 5)


In [18]:
# To send inputs (Mostly Tensors) to GPU as well
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


# Preprocessing

Load Pretrained GloVe Embeddings

In [7]:
glove_embeddings_dict = {}
with open("glove.42B.300d.txt") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        glove_embeddings_dict[word] = vector

# unk_300d is a representation for Unknown tokens
unk_300d = "0.22418134 -0.28881392 0.13854356 0.00365387 -0.12870757 0.10243822 0.061626635 0.07318011 -0.061350107 -1.3477012 0.42037755 -0.063593924 -0.09683349 0.18086134 0.23704372 0.014126852 0.170096 -1.1491593 0.31497982 0.06622181 0.024687296 0.076693475 0.13851812 0.021302193 -0.06640582 -0.010336159 0.13523154 -0.042144544 -0.11938788 0.006948221 0.13333307 -0.18276379 0.052385733 0.008943111 -0.23957317 0.08500333 -0.006894406 0.0015864656 0.063391194 0.19177166 -0.13113557 -0.11295479 -0.14276934 0.03413971 -0.034278486 -0.051366422 0.18891625 -0.16673574 -0.057783455 0.036823478 0.08078679 0.022949161 0.033298038 0.011784158 0.05643189 -0.042776518 0.011959623 0.011552498 -0.0007971594 0.11300405 -0.031369694 -0.0061559738 -0.009043574 -0.415336 -0.18870236 0.13708843 0.005911723 -0.113035575 -0.030096142 -0.23908928 -0.05354085 -0.044904727 -0.20228513 0.0065645403 -0.09578946 -0.07391877 -0.06487607 0.111740574 -0.048649278 -0.16565254 -0.052037314 -0.078968436 0.13684988 0.0757494 -0.006275573 0.28693774 0.52017444 -0.0877165 -0.33010918 -0.1359622 0.114895485 -0.09744406 0.06269521 0.12118575 -0.08026362 0.35256687 -0.060017522 -0.04889904 -0.06828978 0.088740796 0.003964443 -0.0766291 0.1263925 0.07809314 -0.023164088 -0.5680669 -0.037892066 -0.1350967 -0.11351585 -0.111434504 -0.0905027 0.25174105 -0.14841858 0.034635577 -0.07334565 0.06320108 -0.038343467 -0.05413284 0.042197507 -0.090380974 -0.070528865 -0.009174437 0.009069661 0.1405178 0.02958134 -0.036431845 -0.08625681 0.042951006 0.08230793 0.0903314 -0.12279937 -0.013899368 0.048119213 0.08678239 -0.14450377 -0.04424887 0.018319942 0.015026873 -0.100526 0.06021201 0.74059093 -0.0016333034 -0.24960588 -0.023739101 0.016396184 0.11928964 0.13950661 -0.031624354 -0.01645025 0.14079992 -0.0002824564 -0.08052984 -0.0021310581 -0.025350995 0.086938225 0.14308536 0.17146006 -0.13943303 0.048792403 0.09274929 -0.053167373 0.031103406 0.012354865 0.21057427 0.32618305 0.18015954 -0.15881181 0.15322933 -0.22558987 -0.04200665 0.0084689725 0.038156632 0.15188617 0.13274793 0.113756925 -0.095273495 -0.049490947 -0.10265804 -0.27064866 -0.034567792 -0.018810693 -0.0010360252 0.10340131 0.13883452 0.21131058 -0.01981019 0.1833468 -0.10751636 -0.03128868 0.02518242 0.23232952 0.042052146 0.11731903 -0.15506615 0.0063580726 -0.15429358 0.1511722 0.12745973 0.2576985 -0.25486213 -0.0709463 0.17983761 0.054027 -0.09884228 -0.24595179 -0.093028545 -0.028203879 0.094398156 0.09233813 0.029291354 0.13110267 0.15682974 -0.016919162 0.23927948 -0.1343307 -0.22422817 0.14634751 -0.064993896 0.4703685 -0.027190214 0.06224946 -0.091360025 0.21490277 -0.19562101 -0.10032754 -0.09056772 -0.06203493 -0.18876675 -0.10963594 -0.27734384 0.12616494 -0.02217992 -0.16058226 -0.080475815 0.026953284 0.110732645 0.014894041 0.09416802 0.14299914 -0.1594008 -0.066080004 -0.007995227 -0.11668856 -0.13081996 -0.09237365 0.14741232 0.09180138 0.081735 0.3211204 -0.0036552632 -0.047030564 -0.02311798 0.048961394 0.08669574 -0.06766279 -0.50028914 -0.048515294 0.14144728 -0.032994404 -0.11954345 -0.14929578 -0.2388355 -0.019883996 -0.15917352 -0.052084364 0.2801028 -0.0029121689 -0.054581646 -0.47385484 0.17112483 -0.12066923 -0.042173345 0.1395337 0.26115036 0.012869649 0.009291686 -0.0026459037 -0.075331464 0.017840583 -0.26869613 -0.21820338 -0.17084768 -0.1022808 -0.055290595 0.13513643 0.12362477 -0.10980586 0.13980341 -0.20233242 0.08813751 0.3849736 -0.10653763 -0.06199595 0.028849555 0.03230154 0.023856193 0.069950655 0.19310954 -0.077677034 -0.144811"
unk_300d = np.array([float(num) for num in unk_300d.split()])

Save a set of Stopwords

In [8]:
stop_words = set(stopwords.words('english'))

Preprocessing for getting GloVe Embeddings

In [9]:
def preprocessor_glove(s):
  s = s.lower()

  exclude = set(string.punctuation)
  s = ''.join(ch if ch not in exclude else ' ' for ch in s)

  wlist = s.split()

  return wlist

Set up Tokenization (BERT)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Function to find Target token positions in sentence token-list

In [11]:
def tok2pos(sent_toklist, tw):
  currparse_pos = 0
  curr_subset = False

  poslist = []

  for i,tok in enumerate(sent_toklist):  
    if not curr_subset and tw.startswith(tok):
      if tw == tok and ( i+1 == len(sent_toklist) or not (sent_toklist[i+1]).startswith('##')):
        poslist.append(i)
        break
      else:
        poslist.append(i)
        currparse_pos = len(tok)
        curr_subset = True
    
    elif curr_subset and not tok.startswith('##'):
      currparse_pos = 0
      curr_subset = False
      poslist = []
      if tw.startswith(tok):
        if tw == tok and ( i+1 == len(sent_toklist) or not (sent_toklist[i+1]).startswith('##')):
          poslist.append(i)
          break
        else:
          poslist.append(i)
          currparse_pos = len(tok)
          curr_subset = True

    elif curr_subset and tok.startswith('##') and (tw[currparse_pos:]).startswith(tok[2:]):
      if tw[currparse_pos:] == tok[2:] and ( i+1 == len(sent_toklist) or not (sent_toklist[i+1]).startswith('##')):
        poslist.append(i)
        break
      else:
        poslist.append(i)
        currparse_pos += len(tok)-2
    
  return poslist

In [14]:
def find_twpos(df):
  sentlist = list(df['sentence'])
  twordlist = list(df['token'])
  poslist_list = []
  maxlen = 0
  i = 0
  for tw, sent in zip(twordlist, sentlist):
    sent_toklist = tokenizer.tokenize(sent)
    
    poslist = tok2pos(sent_toklist, tw.lower())
    
    if poslist==[]:
      print(i,tw,sent)
      # Take central position of sentence if target word not found 
      poslist_list.append([len(sent_toklist)//2])
    
    else:
      poslist_list.append(poslist)
    
    maxlen = max(maxlen, len(sent_toklist))
    i += 1
  
  print("Max:", maxlen)
  df['target_tokpos'] = poslist_list

In [15]:
find_twpos(traindf)
find_twpos(trialdf)
find_twpos(testdf)

5079 N Superficial and deep anterior cortical staining was grossly different between alphaA/BKO (Figure 3A,3D,3F and 3I) and wild type lenses (Figure 3K,3N,3P and 3S).
Max: 302
Max: 170
522 P Superficial and deep anterior cortical staining was grossly different between alphaA/BKO (Figure 3A,3D,3F and 3I) and wild type lenses (Figure 3K,3N,3P and 3S).
Max: 313


# Feature Extraction

## Extract Various Word Embeddings

#### Extract Contextual BERT embeddings

In [ ]:
bertmodel = BertModel.from_pretrained('bert-base-uncased')
bertmodel.eval()
bertmodel.cuda()

In [19]:
def getwordfeat_bert(df):
  sentlist = list(df['sentence'])
  twordlist = list(df['token'])
  twposlist_list = list(df['target_tokpos'])

  featvec_list = []
  
  for tw, sent, twposlist in zip(twordlist, sentlist, twposlist_list):
    with torch.no_grad():
      inp_embeds = tokenizer(sent, return_tensors="pt").to(DEVICE)
      out_states = bertmodel(**inp_embeds, output_hidden_states=True)
      hlayers = out_states.hidden_states

      # Based on assumption, last hidden layer is often geared towards a specific classification task the BERT was trained on.... 
      # Choosing the 2nd last layer instead to generalize better
      hlayer_minus2 = hlayers[-2]

      # All layers of BERT vectors are of size 768
      embed_t = torch.zeros(1,768, device=DEVICE)
      word_embed = torch.zeros(768, device=DEVICE)

      # Word embedding = Avg of its constituent subword embeddings
      for pos in twposlist:
        word_embed += hlayer_minus2[0][pos+1]
      word_embed = word_embed / len(twposlist)

      embed_t[0] += word_embed

    featvec_list.append(embed_t)
  
  featmat_t = torch.cat(featvec_list)
  featmat = np.array(featmat_t.detach().cpu())

  return featmat

In [21]:
bertword_train = getwordfeat_bert(traindf)
bertword_trial = getwordfeat_bert(trialdf)
bertword_test = getwordfeat_bert(testdf)

In [22]:
print(list(set(list(np.where(np.isnan(bertword_train))[0]))))
print(list(set(list(np.where(np.isnan(bertword_trial))[0]))))
print(list(set(list(np.where(np.isnan(bertword_test))[0]))))

[]
[]
[]


#### Extract GloVe Word Embeddings

In [28]:
def getwordfeat_glove(df):
  sentlist = list(df['sentence'])
  twordlist = list(df['token'])

  featvec_list = []
  
  for tw, sent in zip(twordlist, sentlist):
    tw = tw.strip().lower()
    toklist = preprocessor_glove(sent)
    
    embed_t = torch.zeros(1,300, device=DEVICE)

    word_glove = torch.zeros(300, device=DEVICE)
    valid_wocc = 0
    
    for k,token in enumerate(toklist):
      if token == tw:
        valid_wocc += 1
        if token in glove_embeddings_dict:
          word_glove += torch.Tensor(glove_embeddings_dict[token]).to(DEVICE)
        else:
          word_glove += torch.Tensor(unk_300d).to(DEVICE)

    if valid_wocc != 0:
      word_glove = word_glove / valid_wocc

    else:
      word_glove += torch.Tensor(unk_300d).to(DEVICE)
    
    embed_t[0] += word_glove
    featvec_list.append(embed_t)
  
  featmat_t = torch.cat(featvec_list)
  featmat = np.array(featmat_t.detach().cpu())

  return featmat

In [29]:
gloveword_train = getwordfeat_glove(traindf)
gloveword_trial = getwordfeat_glove(trialdf)
gloveword_test = getwordfeat_glove(testdf)

In [30]:
print(list(set(list(np.where(np.isnan(gloveword_train))[0]))))
print(list(set(list(np.where(np.isnan(gloveword_trial))[0]))))
print(list(set(list(np.where(np.isnan(gloveword_test))[0]))))

[]
[]
[]


## Extract Various Sentence Embeddings

#### Pooled Output (Averaging 2 methods)

In [31]:
def getsentfeat_bert(df):
  sentlist = list(df['sentence'])
  twordlist = list(df['token'])

  featvec_list = []
  
  for tw, sent in zip(twordlist, sentlist):
    tw = tw.strip().lower()
    
    with torch.no_grad():
      inp_embeds = tokenizer(sent, return_tensors="pt").to(DEVICE)
      out_states = bertmodel(**inp_embeds, output_hidden_states=True)
      hlayers = out_states.hidden_states

      # Based on assumption, last hidden layer is often geared towards a specific classification task the BERT was trained on.... 
      # Choosing the 2nd last layer instead to generalize better
      hlayer_minus2 = hlayers[-2]

      embed_t = torch.zeros(1,768, device=DEVICE)
      
      pooled_embed = torch.zeros(768, device=DEVICE)

      for w_e in hlayer_minus2[0]:
         pooled_embed += w_e
      pooled_embed = pooled_embed/((hlayer_minus2[0].shape)[0])

      # Also taking the Standard BERT-Pooler output, ie, last 4-layers CLS combination
      pooled_embed += out_states.pooler_output[0]
      pooled_embed = pooled_embed / 2

      embed_t[0] += pooled_embed

    featvec_list.append(embed_t)
  
  featmat_t = torch.cat(featvec_list)
  featmat = np.array(featmat_t.detach().cpu())

  return featmat

In [32]:
bertsent_train = getsentfeat_bert(traindf)
bertsent_trial = getsentfeat_bert(trialdf)
bertsent_test = getsentfeat_bert(testdf)

In [33]:
print(list(set(list(np.where(np.isnan(bertsent_train))[0]))))
print(list(set(list(np.where(np.isnan(bertsent_trial))[0]))))
print(list(set(list(np.where(np.isnan(bertsent_test))[0]))))

[]
[]
[]


#### Average GloVe

In [34]:
def getsentfeat_glove(df):
  sentlist = list(df['sentence'])
  twordlist = list(df['token'])

  featvec_list = []
  missing = 0
  total = 0
  
  for tw, sent in zip(twordlist, sentlist):
    embed_t = torch.zeros(1,300, device=DEVICE)
    
    toklist = preprocessor_glove(sent)
      
    avg_glove = torch.zeros(300, device=DEVICE)
      
    valid_tcount = 0
    
    for k,token in enumerate(toklist):
      total += 1
      if token in glove_embeddings_dict:
        avg_glove += torch.Tensor(glove_embeddings_dict[token]).to(DEVICE)
        valid_tcount += 1
      else:
        missing += 1

    if valid_tcount == 0:
      print("what")
    
    else:
      avg_glove = avg_glove/valid_tcount
      
    embed_t[0] += avg_glove
    
    featvec_list.append(embed_t)
  

  print("Missing:", missing, "out of", total)
  
  featmat_t = torch.cat(featvec_list)
  featmat = np.array(featmat_t.detach().cpu())

  return featmat

In [36]:
glovesent_train = getsentfeat_glove(traindf)
glovesent_trial = getsentfeat_glove(trialdf)
glovesent_test = getsentfeat_glove(testdf)

Missing: 1667 out of 212261
Missing: 91 out of 11922
Missing: 187 out of 25622


In [37]:
print(list(set(list(np.where(np.isnan(glovesent_train))[0]))))
print(list(set(list(np.where(np.isnan(glovesent_trial))[0]))))
print(list(set(list(np.where(np.isnan(glovesent_test))[0]))))

[]
[]
[]


#### Paraphrase-DistilRoberta Features

In [ ]:
roberta_sentmodel = SentenceTransformer('paraphrase-distilroberta-base-v1')
roberta_sentmodel.eval()
roberta_sentmodel.cuda()

In [39]:
def getsentfeat_dtrob(df):
  sentlist = list(df['sentence'])
  featvec_list = []
  
  for sent in sentlist:
    with torch.no_grad():
      embed_t = torch.zeros(1,768, device=DEVICE)
      sent_embed = roberta_sentmodel.encode(sent, device=DEVICE, convert_to_tensor=True)
      embed_t[0] += sent_embed

    featvec_list.append(embed_t)
  
  featmat_t = torch.cat(featvec_list)
  featmat = np.array(featmat_t.detach().cpu())

  return featmat

In [40]:
dtrobsent_train = getsentfeat_dtrob(traindf)
dtrobsent_trial = getsentfeat_dtrob(trialdf)
dtrobsent_test = getsentfeat_dtrob(testdf)

In [41]:
print(list(set(list(np.where(np.isnan(dtrobsent_train))[0]))))
print(list(set(list(np.where(np.isnan(dtrobsent_trial))[0]))))
print(list(set(list(np.where(np.isnan(dtrobsent_test))[0]))))

[]
[]
[]


## Extract Manual Features

In [42]:
trainmf = traindf.copy()
trialmf = trialdf.copy()
testmf = testdf.copy()

In [43]:
# Add Corpus type information (one-hot)
one_hot_corp_train = pd.get_dummies(trainmf['corpus'])
one_hot_corp_trial = pd.get_dummies(trialmf['subcorpus'])
one_hot_corp_test = pd.get_dummies(testmf['corpus'])

trainmf = trainmf.join(one_hot_corp_train)
trialmf = trialmf.join(one_hot_corp_trial)
testmf = testmf.join(one_hot_corp_test)

In [44]:
# Given a sentence, returns number of vowels
def num_vowels(sentence):
  count=0
  for c in sentence:
    if (c=='a' or c=='e' or c=='i' or c=='o' or c=='u'):
      count+=1
  return count

In [45]:
def remove_punc(sen):
  x = re.compile('['+string.punctuation+']'+'–')
  new_sen = re.sub(x, ' ', sen)
  return new_sen

#### POS Tag Extraction

In [ ]:
# Get all the pos_tags possible from the training corpus
pos_tag_dic={}
pos_result=set()
pos_result.add('None')

In [46]:
def pos_tagger_old(txt):
  txt = remove_punc(txt)
  tokenized = sent_tokenize(txt)
  
  for i in tokenized:
    # Word tokenizers is used to find the words
    # and punctuation in a string
    wordsList = nltk.word_tokenize(i)

    # removing stop words from wordList
    wordsList = [w for w in wordsList if not w in stop_words]

    # Using a Tagger. Which is part-of-speech
    # tagger or POS-tagger.
    tagged = nltk.pos_tag(wordsList)

    count = Counter([j for i,j in tagged])
    for i in count:
      if i in pos_tag_dic.keys():
        pos_tag_dic[i]+=count[i]
      else:
        pos_tag_dic[i]=count[i]

In [47]:
for i in range(len(trainmf['sentence'])):
  pos_tagger_old(trainmf['sentence'][i])

#add a feature for each pos_tag
for i in pos_tag_dic.keys():
  trainmf["c_"+i]=[0]*len(trainmf)

In [48]:
trainmf["tw_postag"]=["None"]*len(trainmf)

In [ ]:
def pos_tagger_train(row,txt,tw,df):
  txt = remove_punc(txt)
  tokenized = sent_tokenize(txt)
  
  for i in tokenized:  
    # Word tokenizers is used to find the words
    # and punctuation in a string
    wordsList = nltk.word_tokenize(i)

    # removing stop words from wordList
    wordsList = [w for w in wordsList if not w in stop_words]

    # Using a Tagger. Which is part-of-speech
    # tagger or POS-tagger.
    tagged = nltk.pos_tag(wordsList)

    for i,j in tagged:
      if i.lower() == tw.lower():
        df['tw_postag'][row]=j
        pos_result.add(j)
    count = Counter([j for i,j in tagged])

    new_pos_tag_dic={}
    for k in pos_tag_dic.keys():
      new_pos_tag_dic[k]=0
    for i in count:
      if i in new_pos_tag_dic.keys():
        new_pos_tag_dic[i]+=count[i]
    
    for i in new_pos_tag_dic.keys():
      df["c_"+i][row]=new_pos_tag_dic[i]

In [ ]:
for i in range(len(trainmf)):
  pos_tagger_train(i,trainmf['sentence'][i],trainmf['token'][i],trainmf)

In [50]:
#getting the columns which has more than 90% data as zeros
x = list(zip(trainmf.columns,list(map(lambda i : (0.1*len(trainmf)>i),trainmf.astype(bool).sum(axis=0)))))
for name,val in x:
  if val:
    trainmf.drop(name,axis=1,inplace=True)
    del pos_tag_dic[name[2:]]

In [51]:
for k in pos_tag_dic.keys():
  trialmf["c_"+k]=[0]*len(trialmf)
  testmf["c_"+k]=[0]*len(testmf)

for i in pos_result:
  testmf[i]=[0]*len(testmf)
  trialmf[i]=[0]*len(trialmf)

In [52]:
one_hot_pos = pd.get_dummies(trainmf['tw_postag'])
# Drop column as it is now encoded
trainmf = trainmf.drop('tw_postag',axis = 1)
# Join the encoded df
trainmf = trainmf.join(one_hot_pos)

In [53]:
def pos_tagger_test(row,txt,tw,df):
  txt = remove_punc(txt)
  tokenized = sent_tokenize(txt)
  for i in tokenized:
    
    # Word tokenizers is used to find the words
    # and punctuation in a string
    wordsList = nltk.word_tokenize(i)

    # removing stop words from wordList
    wordsList = [w for w in wordsList if not w in stop_words]

    # Using a Tagger. Which is part-of-speech
    # tagger or POS-tagger.
    tagged = nltk.pos_tag(wordsList)

    for i,j in tagged:
      if i.lower() == tw.lower() and j in pos_result:
        if j not in df.columns:
          df[j]=[0]*len(df)
        df[j][row]=1
    count = Counter([j for i,j in tagged])

    new_pos_tag_dic={}
    for k in pos_tag_dic.keys():
        new_pos_tag_dic[k]=0
    for i in count:
      if i in new_pos_tag_dic.keys():
        new_pos_tag_dic[i]+=count[i]
    
    for i in new_pos_tag_dic.keys():
      df["c_"+i][row]=new_pos_tag_dic[i]

In [ ]:
for i in range(len(testmf)):
  pos_tagger_test(i,testmf['sentence'][i],testmf['token'][i],testmf)

for i in range(len(trialmf)):
  pos_tagger_test(i,trialmf['sentence'][i],trialmf['token'][i],trialmf)

#### Readability Features Extraction

In [56]:
# Load Easy word set from Textstat
easy_word_set = textstat._textstatistics__get_lang_easy_words()

# Load pretrained SpaCy model
nlp = spacy.load('en')

In [57]:
# Splits the text into sentences, using
# Spacy's sentence segmentation
def break_sentences(text):
	doc = nlp(text)
	return doc.sents

# Returns Number of Words in the text
def word_count(text):
	sentences = break_sentences(text)
	words = 0
	for sentence in sentences:
		words += len([token for token in sentence])
	return words

# Returns the number of sentences in the text
def sentence_count(text):
  sentences = break_sentences(text)
  return len(list(sentences))


# Returns average sentence length
def avg_sentence_length(text):
	words = word_count(text)
	sentences = sentence_count(text)
	average_sentence_length = float(words / sentences)
	return average_sentence_length

# Using Textstat to calculate statistics from text to 
# determine readability, complexity and grade level of a particular corpus.
def syllables_count(word):
	return textstatistics().syllable_count(str(word))

# Returns the average number of syllables per
# word in the text
def avg_syllables_per_word(text):
	syllable = syllables_count(text)
	words = word_count(text)
	ASPW = float(syllable) / float(words)
	return legacy_round(ASPW, 1)

# Return total Difficult Words in a text
def difficult_words(text):

	# Find all words in the text
	words = []
	sentences = break_sentences(text)
	for sentence in sentences:
		words += [str(token) for token in sentence]

	# Difficult words are those with syllables >= 2
	# easy_word_set is provide by Textstat as
	# a list of common words
	diff_words_set = set()
	
	for word in words:
		syllable_count = syllables_count(word)
		if word not in easy_word_set and syllable_count >= 2:
			diff_words_set.add(word)

	return len(diff_words_set)

# A word is polysyllablic if it has more than 3 syllables
# this functions returns the number of all such words
# present in the text
def poly_syllable_count(text):
	count = 0
	words = []
	sentences = break_sentences(text)
	for sentence in sentences:
		words += [token for token in sentence]
	

	for word in words:
		syllable_count = syllables_count(word)
		if syllable_count >= 3:
			count += 1
	return count


def flesch_reading_ease(text):
	"""
		Implements Flesch Formula:
		Reading Ease score = 206.835 - (1.015 × ASL) - (84.6 × ASW)
		Here,
		ASL = average sentence length (number of words
				divided by number of sentences)
		ASW = average word length in syllables (number of syllables
				divided by number of words)
	"""
	FRE = 206.835 - float(1.015 * avg_sentence_length(text)) -\
		float(84.6 * avg_syllables_per_word(text))
	return legacy_round(FRE, 2)


def gunning_fog(text):
	per_diff_words = (difficult_words(text) / word_count(text) * 100) + 5
	grade = 0.4 * (avg_sentence_length(text) + per_diff_words)
	return grade


def smog_index(text):
	"""
		Implements SMOG Formula / Grading
		SMOG grading = 3 + ?polysyllable count.
		Here,
		polysyllable count = number of words of more
		than two syllables in a sample of 30 sentences.
	"""

	if sentence_count(text) >= 3:
		poly_syllab = poly_syllable_count(text)
		SMOG = (1.043 * (30*(poly_syllab / sentence_count(text)))**0.5) \
				+ 3.1291
		return legacy_round(SMOG, 1)
	else:
		return 0


def dale_chall_readability_score(text):
	"""
		Implements Dale Challe Formula:
		Raw score = 0.1579*(PDW) + 0.0496*(ASL) + 3.6365
		Here,
			PDW = Percentage of difficult words.
			ASL = Average sentence length
	"""
	words = word_count(text)
	# Number of words not termed as difficult words
	count = words - difficult_words(text)
	if words > 0:

		# Percentage of words not on difficult word list

		per = float(count) / float(words) * 100
	
	# diff_words stores percentage of difficult words
	diff_words = 100 - per

	raw_score = (0.1579 * diff_words) + \
				(0.0496 * avg_sentence_length(text))
	
	# If Percentage of Difficult Words is greater than 5 %, then;
	# Adjusted Score = Raw Score + 3.6365,
	# otherwise Adjusted Score = Raw Score

	if diff_words > 5:	

		raw_score += 3.6365
		
	return legacy_round(raw_score, 2)

In [59]:
# Function add the different features
def add_feature(name,feature,df,for_sen,for_tw):
  if for_sen:
    ls = []
    for i in range(len(df['sentence'])):
      ls.append(feature(df['sentence'][i]))
    df[name+'_sent']=ls
  if for_tw:
    ls = []
    for i in range(len(df['token'])):
      ls.append(feature(df['token'][i]))
    df[name+'_tw']=ls

In [60]:
add_feature("vowel_c",num_vowels,trainmf,1,1)
add_feature("dale_chall_readability_score",dale_chall_readability_score,trainmf,1,1)
add_feature("smog_index",smog_index,trainmf,1,0)
add_feature("gunning_fog",gunning_fog,trainmf,1,0)
add_feature("flesch_reading_ease",flesch_reading_ease,trainmf,1,0)

add_feature("vowel_c",num_vowels,trialmf,1,1)
add_feature("dale_chall_readability_score",dale_chall_readability_score,trialmf,1,1)
add_feature("smog_index",smog_index,trialmf,1,0)
add_feature("gunning_fog",gunning_fog,trialmf,1,0)
add_feature("flesch_reading_ease",flesch_reading_ease,trialmf,1,0)

add_feature("vowel_c",num_vowels,testmf,1,1)
add_feature("dale_chall_readability_score",dale_chall_readability_score,testmf,1,1)
add_feature("smog_index",smog_index,testmf,1,0)
add_feature("gunning_fog",gunning_fog,testmf,1,0)
add_feature("flesch_reading_ease",flesch_reading_ease,testmf,1,0)

#### Drop Other columns, Save Features as Normalized Numpy Array

In [61]:
trainmf.drop(['id', 'corpus', 'sentence', 'token', 'complexity', 'target_tokpos'],axis=1,inplace=True)
trialmf.drop(['id', 'subcorpus', 'sentence', 'token', 'complexity', 'target_tokpos'],axis=1,inplace=True)
testmf.drop(['id', 'corpus', 'sentence', 'token', 'complexity', 'target_tokpos'],axis=1,inplace=True)

In [62]:
trainmf = trainmf.to_numpy()
trialmf = trialmf.to_numpy()
testmf = testmf.to_numpy()

# Feature Scaling Normalization for Manual Features
norm = MinMaxScaler().fit(trainmf)

trainmf = norm.transform(trainmf)
trialmf = norm.transform(trialmf)
testmf = norm.transform(testmf)

## Check Dimensions of all Extracted Features

In [64]:
print('Dataset Size (Extra Column added for Token-Position) :')
print(traindf.shape)
print(trialdf.shape)
print(testdf.shape)
print('------------------------------------------------------')
print('Size of Extracted Features :')
print('---------------')
print('Manual Featues :')
print(trainmf.shape)
print(trialmf.shape)
print(testmf.shape)
print('---------------')
print('BERT Contextual Word-Embeddings :')
print(bertword_train.shape)
print(bertword_trial.shape)
print(bertword_test.shape)
print('---------------')
print('GloVe Word-Embeddings :')
print(gloveword_train.shape)
print(gloveword_trial.shape)
print(gloveword_test.shape)
print('---------------')
print('BERT Special Pooled Sentence-Embeddings :')
print(bertsent_train.shape)
print(bertsent_trial.shape)
print(bertsent_test.shape)
print('---------------')
print('Paraphrase-DistilRoberta Sentence-Embeddings :')
print(dtrobsent_train.shape)
print(dtrobsent_trial.shape)
print(dtrobsent_test.shape)
print('---------------')
print('GloVe-Averaged Sentence-Embeddings :')
print(glovesent_train.shape)
print(glovesent_trial.shape)
print(glovesent_test.shape)

Dataset Size (Extra Column added for Token-Position) :
(7662, 6)
(421, 6)
(917, 6)
------------------------------------------------------
Size of Extracted Features :
---------------
Manual Featues :
(7662, 55)
(421, 55)
(917, 55)
---------------
BERT Contextual Word-Embeddings :
(7662, 768)
(421, 768)
(917, 768)
---------------
GloVe Word-Embeddings :
(7662, 300)
(421, 300)
(917, 300)
---------------
BERT Special Pooled Sentence-Embeddings :
(7662, 768)
(421, 768)
(917, 768)
---------------
Paraphrase-DistilRoberta Sentence-Embeddings :
(7662, 768)
(421, 768)
(917, 768)
---------------
GloVe-Averaged Sentence-Embeddings :
(7662, 300)
(421, 300)
(917, 300)


# Base Models

## Fully-Connected Neural Networks

### Dataloader, Datasets, Evaluation, & Training Function for Torch NN

In [65]:
def mae(y_pred,probability):
  mae_v = 0
  for i,row in enumerate(y_pred):
    mae_v += abs(row[0]-probability[i][0])
  mae_v = mae_v/len(y_pred)
  return mae_v

In [66]:
def pearson(y_pred, label):
  if y_pred.device.type == 'cuda':
    y_pred_np = np.array(y_pred.detach().cpu())  
  else:
    y_pred_np = np.array(y_pred)

  if label.device.type == 'cuda':
    label_np = np.array(label.cpu())
  else:
    label_np = np.array(label)

  label_np = label_np.reshape(-1)
  y_pred_np = y_pred_np.reshape(-1)

  pcoff, _ = stats.pearsonr(y_pred_np, label_np)

  return pcoff

In [67]:
class dataset(Dataset):
  def __init__(self,data,labels):
    self.data = data
    self.labels = labels

  def __getitem__(self, index):
    return self.data[index], self.labels[index]
  
  def __len__(self):
    return len(self.labels)

In [68]:
def my_collate_fn(batch):
  return [x[0] for x in batch],[x[1] for x in batch]

Custom Evaluation (Testing) Function:

In [69]:
def testeval(model, data_loader, DEVICE):
  testiter = iter(data_loader)
  testd_ = testiter.next()
  data_tst, labs_tst = testd_

  labs_tst = torch.Tensor(labs_tst).to(DEVICE)
  labs_tst = torch.unsqueeze(labs_tst, 1)

  ypreds_tst = model(data_tst)

  #MAE
  mae_v = mae(ypreds_tst, labs_tst).detach().cpu()
  #MSE
  loss = nn.MSELoss()
  mse_v = loss(ypreds_tst, labs_tst).detach().cpu()
  #Pearson Coefficient
  pcoff_v = pearson(ypreds_tst, labs_tst)
  
  print("[ MAE value: ", mae_v, " ]\t[ MSE value: ", mse_v, " ]\t[ Pearson Coefficient: ", pcoff_v, " ]")

  return pcoff_v

Custom Training function for Torch NNs:

In [70]:
def train(model, tr_data, tr_lab, val_data, val_lab, tst_data, tst_lab, MAX_EPOCHS, BATCH_SIZE, LEARNING_RATE, DEVICE):
  iters_per_epoch = math.ceil(len(tr_lab)/BATCH_SIZE)
  VALBATCH_SIZE = len(val_lab)
  TESTBATCH_SIZE = len(tst_lab)

  train_data = dataset(tr_data, tr_lab)
  valid_data = dataset(val_data, val_lab)
  test_data = dataset(tst_data, tst_lab)

  train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=my_collate_fn)

  train_loader_full = DataLoader(dataset=train_data, batch_size=len(tr_lab), shuffle=False, collate_fn=my_collate_fn)
  valid_loader = DataLoader(dataset=valid_data, batch_size=VALBATCH_SIZE, shuffle=False, collate_fn=my_collate_fn)
  test_loader = DataLoader(dataset=test_data, batch_size=TESTBATCH_SIZE, shuffle=False, collate_fn=my_collate_fn)

  criterion = (nn.MSELoss()).to(DEVICE)
  optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

  for i in range(1,MAX_EPOCHS+1):
    print("[Epoch: ",str(i),"/",str(MAX_EPOCHS),"]")
    
    model.train()
    
    for j,(data_batch, label_batch) in enumerate(tqdm(train_loader, total=iters_per_epoch)):
      label_batch = torch.Tensor(label_batch).to(DEVICE)
      label_batch = torch.unsqueeze(label_batch, 1)

      optimizer.zero_grad()

      y_pred_batch = model(data_batch)
      
      loss = criterion(y_pred_batch, label_batch)
      
      loss.backward()
      optimizer.step()
    
    model.eval()
    print('Validation scores: ', end='')
    curr_pscore = testeval(model, valid_loader, DEVICE)
    print('---------------------')
    print('Test scores: ', end='')
    curr_pscore = testeval(model, test_loader, DEVICE)
    print('---------------------')
    print('Train scores: ', end='')
    testeval(model, train_loader_full, DEVICE)
    print('------------------------------------------------------------------------------------------------------')

### FCNN v1 Framework (3 hidden; relu 1st 2 ; sigmoid last) (Default Dropouts 0.5, 0.25, 0.125)

In [71]:
class Framework_v1(nn.Module):
    def __init__(self, in_size, h1_size, h2_size, h3_size, DEVICE, drop_1=0.5, drop_2=0.25, drop_3=0.125):
        super().__init__()
        self.device = DEVICE

        self.layer_1 = (nn.Linear(in_size,h1_size)).to(DEVICE)
        self.layer_2 = (nn.Linear(h1_size,h2_size)).to(DEVICE)
        self.layer_3 = (nn.Linear(h2_size,h3_size)).to(DEVICE)
        self.layer_out = (nn.Linear(h3_size,1)).to(DEVICE)
        self.layer_drop1 = (nn.Dropout(drop_1)).to(DEVICE)
        self.layer_drop2 = (nn.Dropout(drop_2)).to(DEVICE)
        self.layer_drop3 = (nn.Dropout(drop_3)).to(DEVICE)
        
        # Define activations
        self.sigmoid = nn.Sigmoid().to(DEVICE)
        self.relu = nn.ReLU().to(DEVICE)
    
    
    def lasthidden(self, x):
        # Pass the input tensor through each of our operations (except dropouts & output layer)
        x = torch.Tensor(x).to(self.device)

        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_3(out)
        out = self.sigmoid(out)

        return out
        

    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = torch.Tensor(x).to(self.device)

        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_drop1(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_drop2(out)
        out = self.layer_3(out)
        out = self.sigmoid(out)
        out = self.layer_drop3(out)
        out = self.layer_out(out)
        out = self.sigmoid(out)

        return out

### Training & Testing FCNNs

##### Case-1 - 0.75535 - (bertword_train,dtrobsent_train,gloveword_train,glovesent_train)

In [118]:
# Define Model
fcnnf1_1 = Framework_v1(2136,1024,512,256,DEVICE,0.4,0.4,0.4)

# Train Model with selected hyperparams
MAX_EPOCHS = 20
BATCH_SIZE = 16
LEARNING_RATE = 1e-5

featmat_train = np.concatenate((bertword_train,dtrobsent_train,gloveword_train,glovesent_train),axis=1)
featmat_val = np.concatenate((bertword_trial,dtrobsent_trial,gloveword_trial,glovesent_trial),axis=1)
featmat_test = np.concatenate((bertword_test,dtrobsent_test,gloveword_test,glovesent_test),axis=1)

labels_train = np.array(traindf['complexity'])
labels_val = np.array(trialdf['complexity'])
labels_test = np.array(testdf['complexity'])

train(fcnnf1_1, featmat_train, labels_train, featmat_val, labels_val, featmat_test, labels_test, MAX_EPOCHS, BATCH_SIZE, LEARNING_RATE, DEVICE)

  3%|▎         | 16/479 [00:00<00:02, 155.14it/s]

[Epoch:  1 / 20 ]


100%|██████████| 479/479 [00:03<00:00, 152.69it/s]


Validation scores: [ MAE value:  tensor(0.0908)  ]	[ MSE value:  tensor(0.0148)  ]	[ Pearson Coefficient:  0.5850055126792502  ]
---------------------
Test scores: [ MAE value:  tensor(0.0883)  ]	[ MSE value:  tensor(0.0131)  ]	[ Pearson Coefficient:  0.5108875596559543  ]
---------------------
Train scores: 

  3%|▎         | 16/479 [00:00<00:02, 154.53it/s]

...

Validation scores: [ MAE value:  tensor(0.0655)  ]	[ MSE value:  tensor(0.0073)  ]	[ Pearson Coefficient:  0.7848963734673352  ]
---------------------
Test scores: [ MAE value:  tensor(0.0652)  ]	[ MSE value:  tensor(0.0071)  ]	[ Pearson Coefficient:  0.7542339264170366  ]
---------------------
Train scores: 

  3%|▎         | 16/479 [00:00<00:02, 156.99it/s]

[ MAE value:  tensor(0.0510)  ]	[ MSE value:  tensor(0.0042)  ]	[ Pearson Coefficient:  0.8724796767886899  ]
------------------------------------------------------------------------------------------------------
[Epoch:  20 / 20 ]


100%|██████████| 479/479 [00:03<00:00, 153.96it/s]


Validation scores: [ MAE value:  tensor(0.0658)  ]	[ MSE value:  tensor(0.0074)  ]	[ Pearson Coefficient:  0.7826258486981806  ]
---------------------
Test scores: [ MAE value:  tensor(0.0651)  ]	[ MSE value:  tensor(0.0071)  ]	[ Pearson Coefficient:  0.7553539325815444  ]
---------------------
Train scores: [ MAE value:  tensor(0.0503)  ]	[ MSE value:  tensor(0.0041)  ]	[ Pearson Coefficient:  0.8767810562868135  ]
------------------------------------------------------------------------------------------------------


In [121]:
predf1_train = np.array(fcnnf1_1.forward(np.concatenate((bertword_train,dtrobsent_train,gloveword_train,glovesent_train),axis=1)).detach().cpu()).reshape(-1)
predf1 = np.array(fcnnf1_1.forward(np.concatenate((bertword_test,dtrobsent_test,gloveword_test,glovesent_test),axis=1)).detach().cpu()).reshape(-1)
predf1_trial = np.array(fcnnf1_1.forward(np.concatenate((bertword_trial,dtrobsent_trial,gloveword_trial,glovesent_trial),axis=1)).detach().cpu()).reshape(-1)

In [122]:
stats.pearsonr(predf1, testdf['complexity'])

(0.7553539343795164, 3.941816568322594e-170)

##### Case-2 - 0.7574 - (bertword_train,bertsent_train,gloveword_train,glovesent_train)

In [ ]:
# Define Model
fcnnf2 = Framework_v1(2136,1024,512,256,DEVICE,0.4,0.4,0.4)

# Train Model with selected hyperparams
MAX_EPOCHS = 20
BATCH_SIZE = 16
LEARNING_RATE = 1e-5

featmat_train = np.concatenate((bertword_train,bertsent_train,gloveword_train,glovesent_train),axis=1)
featmat_val = np.concatenate((bertword_trial,bertsent_trial,gloveword_trial,glovesent_trial),axis=1)
featmat_test = np.concatenate((bertword_test,bertsent_test,gloveword_test,glovesent_test),axis=1)

labels_train = np.array(traindf['complexity'])
labels_val = np.array(trialdf['complexity'])
labels_test = np.array(testdf['complexity'])

train(fcnnf2, featmat_train, labels_train, featmat_val, labels_val, featmat_test, labels_test, MAX_EPOCHS, BATCH_SIZE, LEARNING_RATE, DEVICE)

In [92]:
# Define Model
fcnnf2_2 = Framework_v1(2136,1024,512,256,DEVICE,0.4,0.4,0.4)

# Train Model with selected hyperparams
MAX_EPOCHS = 25
BATCH_SIZE = 16
LEARNING_RATE = 1e-5

featmat_train = np.concatenate((bertword_train,bertsent_train,gloveword_train,glovesent_train),axis=1)
featmat_val = np.concatenate((bertword_trial,bertsent_trial,gloveword_trial,glovesent_trial),axis=1)
featmat_test = np.concatenate((bertword_test,bertsent_test,gloveword_test,glovesent_test),axis=1)

labels_train = np.array(traindf['complexity'])
labels_val = np.array(trialdf['complexity'])
labels_test = np.array(testdf['complexity'])

train(fcnnf2_2, featmat_train, labels_train, featmat_val, labels_val, featmat_test, labels_test, MAX_EPOCHS, BATCH_SIZE, LEARNING_RATE, DEVICE)

  3%|▎         | 16/479 [00:00<00:02, 157.82it/s]

[Epoch:  1 / 25 ]


100%|██████████| 479/479 [00:03<00:00, 153.43it/s]


Validation scores: [ MAE value:  tensor(0.0936)  ]	[ MSE value:  tensor(0.0155)  ]	[ Pearson Coefficient:  0.5911809144008207  ]
---------------------
Test scores: [ MAE value:  tensor(0.0894)  ]	[ MSE value:  tensor(0.0136)  ]	[ Pearson Coefficient:  0.5266465548307705  ]
---------------------
Train scores: 

  3%|▎         | 12/479 [00:00<00:04, 115.12it/s]

...

Validation scores: [ MAE value:  tensor(0.0661)  ]	[ MSE value:  tensor(0.0074)  ]	[ Pearson Coefficient:  0.7870729625909723  ]
---------------------
Test scores: [ MAE value:  tensor(0.0651)  ]	[ MSE value:  tensor(0.0071)  ]	[ Pearson Coefficient:  0.7571123002126393  ]
---------------------
Train scores: 

  3%|▎         | 16/479 [00:00<00:02, 154.82it/s]

[ MAE value:  tensor(0.0519)  ]	[ MSE value:  tensor(0.0044)  ]	[ Pearson Coefficient:  0.8736981752667619  ]
------------------------------------------------------------------------------------------------------
[Epoch:  25 / 25 ]


100%|██████████| 479/479 [00:03<00:00, 152.92it/s]


Validation scores: [ MAE value:  tensor(0.0658)  ]	[ MSE value:  tensor(0.0071)  ]	[ Pearson Coefficient:  0.7899566515474962  ]
---------------------
Test scores: [ MAE value:  tensor(0.0656)  ]	[ MSE value:  tensor(0.0072)  ]	[ Pearson Coefficient:  0.7574037901160383  ]
---------------------
Train scores: [ MAE value:  tensor(0.0504)  ]	[ MSE value:  tensor(0.0041)  ]	[ Pearson Coefficient:  0.8764848658901985  ]
------------------------------------------------------------------------------------------------------


In [132]:
predf2_train = np.array(fcnnf2_2.forward(np.concatenate((bertword_train,bertsent_train,gloveword_train,glovesent_train),axis=1)).detach().cpu()).reshape(-1)
predf2 = np.array(fcnnf2_2.forward(np.concatenate((bertword_test,bertsent_test,gloveword_test,glovesent_test),axis=1)).detach().cpu()).reshape(-1)
predf2_trial = np.array(fcnnf2_2.forward(np.concatenate((bertword_trial,bertsent_trial,gloveword_trial,glovesent_trial),axis=1)).detach().cpu()).reshape(-1)

In [140]:
stats.pearsonr(predf2, testdf['complexity'])

(0.7574037918021691, 1.4275926358056614e-171)

##### Case-3 - 0.7553 - (May Remove) (bertword_train,bertsent_train,dtrobsent_train,gloveword_train,glovesent_train)

In [91]:
# Define Model
fcnnf3 = Framework_v1(2904,1200,512,256,DEVICE,0.5,0.5,0.5)

# Train Model with selected hyperparams
MAX_EPOCHS = 26
BATCH_SIZE = 16
LEARNING_RATE = 1e-5

featmat_train = np.concatenate((bertword_train,bertsent_train,dtrobsent_train,gloveword_train,glovesent_train),axis=1)
featmat_val = np.concatenate((bertword_trial,bertsent_trial,dtrobsent_trial,gloveword_trial,glovesent_trial),axis=1)
featmat_test = np.concatenate((bertword_test,bertsent_test,dtrobsent_test,gloveword_test,glovesent_test),axis=1)

labels_train = np.array(traindf['complexity'])
labels_val = np.array(trialdf['complexity'])
labels_test = np.array(testdf['complexity'])

train(fcnnf3, featmat_train, labels_train, featmat_val, labels_val, featmat_test, labels_test, MAX_EPOCHS, BATCH_SIZE, LEARNING_RATE, DEVICE)

  2%|▏         | 11/479 [00:00<00:04, 104.15it/s]

[Epoch:  1 / 26 ]


100%|██████████| 479/479 [00:04<00:00, 114.88it/s]


Validation scores: [ MAE value:  tensor(0.0940)  ]	[ MSE value:  tensor(0.0158)  ]	[ Pearson Coefficient:  0.5138673587193796  ]
---------------------
Test scores: [ MAE value:  tensor(0.0904)  ]	[ MSE value:  tensor(0.0139)  ]	[ Pearson Coefficient:  0.44897405146955927  ]
---------------------
Train scores: 

  3%|▎         | 12/479 [00:00<00:03, 116.85it/s]

...

Validation scores: [ MAE value:  tensor(0.0652)  ]	[ MSE value:  tensor(0.0072)  ]	[ Pearson Coefficient:  0.7882638125069839  ]
---------------------
Test scores: [ MAE value:  tensor(0.0654)  ]	[ MSE value:  tensor(0.0072)  ]	[ Pearson Coefficient:  0.7555972070791628  ]
---------------------
Train scores: 

  3%|▎         | 13/479 [00:00<00:03, 121.23it/s]

[ MAE value:  tensor(0.0515)  ]	[ MSE value:  tensor(0.0043)  ]	[ Pearson Coefficient:  0.8704857584947416  ]
------------------------------------------------------------------------------------------------------
[Epoch:  26 / 26 ]


100%|██████████| 479/479 [00:04<00:00, 117.12it/s]


Validation scores: [ MAE value:  tensor(0.0649)  ]	[ MSE value:  tensor(0.0071)  ]	[ Pearson Coefficient:  0.7906904655204966  ]
---------------------
Test scores: [ MAE value:  tensor(0.0655)  ]	[ MSE value:  tensor(0.0072)  ]	[ Pearson Coefficient:  0.7552999219792492  ]
---------------------
Train scores: [ MAE value:  tensor(0.0508)  ]	[ MSE value:  tensor(0.0042)  ]	[ Pearson Coefficient:  0.8738569087401942  ]
------------------------------------------------------------------------------------------------------


In [142]:
predf3_train = np.array(fcnnf3.forward(np.concatenate((bertword_train,bertsent_train, dtrobsent_train, gloveword_train,glovesent_train),axis=1)).detach().cpu()).reshape(-1)
predf3 = np.array(fcnnf3.forward(np.concatenate((bertword_test,bertsent_test, dtrobsent_test, gloveword_test,glovesent_test),axis=1)).detach().cpu()).reshape(-1)
predf3_trial = np.array(fcnnf3.forward(np.concatenate((bertword_trial,bertsent_trial, dtrobsent_trial, gloveword_trial,glovesent_trial),axis=1)).detach().cpu()).reshape(-1)

In [143]:
stats.pearsonr(predf3, testdf['complexity'])

(0.7552999236814216, 4.3000527720487294e-170)

##### Case-4 - 0.7534 - (May Remove) (trainmf,bertword_train,bertsent_train,dtrobsent_train,gloveword_train)

In [109]:
# Define Model
fcnnf4 = Framework_v1(2659,1200,512,256,DEVICE,0.5,0.4,0.3)

# Train Model with selected hyperparams
MAX_EPOCHS = 25
BATCH_SIZE = 16
LEARNING_RATE = 1e-5

featmat_train = np.concatenate((trainmf,bertword_train,bertsent_train,dtrobsent_train,gloveword_train),axis=1)
featmat_val = np.concatenate((trialmf,bertword_trial,bertsent_trial,dtrobsent_trial,gloveword_trial),axis=1)
featmat_test = np.concatenate((testmf,bertword_test,bertsent_test,dtrobsent_test,gloveword_test),axis=1)

labels_train = np.array(traindf['complexity'])
labels_val = np.array(trialdf['complexity'])
labels_test = np.array(testdf['complexity'])

train(fcnnf4, featmat_train, labels_train, featmat_val, labels_val, featmat_test, labels_test, MAX_EPOCHS, BATCH_SIZE, LEARNING_RATE, DEVICE)

  3%|▎         | 15/479 [00:00<00:03, 146.07it/s]

[Epoch:  1 / 25 ]


100%|██████████| 479/479 [00:03<00:00, 142.56it/s]


Validation scores: [ MAE value:  tensor(0.0947)  ]	[ MSE value:  tensor(0.0163)  ]	[ Pearson Coefficient:  0.3827008079309545  ]
---------------------
Test scores: [ MAE value:  tensor(0.0914)  ]	[ MSE value:  tensor(0.0144)  ]	[ Pearson Coefficient:  0.34609342605756704  ]
---------------------
Train scores: 

  3%|▎         | 15/479 [00:00<00:03, 144.32it/s]

...

Validation scores: [ MAE value:  tensor(0.0653)  ]	[ MSE value:  tensor(0.0072)  ]	[ Pearson Coefficient:  0.789200059651128  ]
---------------------
Test scores: [ MAE value:  tensor(0.0655)  ]	[ MSE value:  tensor(0.0072)  ]	[ Pearson Coefficient:  0.7529559896560557  ]
---------------------
Train scores: 

  3%|▎         | 15/479 [00:00<00:03, 141.77it/s]

[ MAE value:  tensor(0.0485)  ]	[ MSE value:  tensor(0.0038)  ]	[ Pearson Coefficient:  0.8859630708514389  ]
------------------------------------------------------------------------------------------------------
[Epoch:  25 / 25 ]


100%|██████████| 479/479 [00:03<00:00, 142.19it/s]


Validation scores: [ MAE value:  tensor(0.0654)  ]	[ MSE value:  tensor(0.0071)  ]	[ Pearson Coefficient:  0.7891672964875625  ]
---------------------
Test scores: [ MAE value:  tensor(0.0652)  ]	[ MSE value:  tensor(0.0072)  ]	[ Pearson Coefficient:  0.7534803465066588  ]
---------------------
Train scores: [ MAE value:  tensor(0.0474)  ]	[ MSE value:  tensor(0.0037)  ]	[ Pearson Coefficient:  0.8904357451716709  ]
------------------------------------------------------------------------------------------------------


In [147]:
predf4_train = np.array(fcnnf4.forward(np.concatenate((trainmf, bertword_train,bertsent_train, dtrobsent_train, gloveword_train),axis=1)).detach().cpu()).reshape(-1)
predf4 = np.array(fcnnf4.forward(np.concatenate((testmf, bertword_test,bertsent_test, dtrobsent_test, gloveword_test),axis=1)).detach().cpu()).reshape(-1)
predf4_trial = np.array(fcnnf4.forward(np.concatenate((trialmf, bertword_trial,bertsent_trial, dtrobsent_trial, gloveword_trial),axis=1)).detach().cpu()).reshape(-1)

In [149]:
stats.pearsonr(predf4, testdf['complexity'])

(0.7534803484191827, 7.951043972916586e-169)

##### Case-5 - 0.75689 - (trainmf,bertword_train,bertsent_train,gloveword_train,glovesent_train)

In [128]:
# Define Model
fcnnf5 = Framework_v1(2191,1024,512,256,DEVICE,0.5,0.4,0.4)

# Train Model with selected hyperparams
MAX_EPOCHS = 25
BATCH_SIZE = 16
LEARNING_RATE = 1e-5

featmat_train = np.concatenate((trainmf,bertword_train,bertsent_train,glovesent_train,gloveword_train),axis=1)
featmat_val = np.concatenate((trialmf,bertword_trial,bertsent_trial,glovesent_trial,gloveword_trial),axis=1)
featmat_test = np.concatenate((testmf,bertword_test,bertsent_test,glovesent_test,gloveword_test),axis=1)

labels_train = np.array(traindf['complexity'])
labels_val = np.array(trialdf['complexity'])
labels_test = np.array(testdf['complexity'])

train(fcnnf5, featmat_train, labels_train, featmat_val, labels_val, featmat_test, labels_test, MAX_EPOCHS, BATCH_SIZE, LEARNING_RATE, DEVICE)

  4%|▍         | 18/479 [00:00<00:02, 171.10it/s]

[Epoch:  1 / 25 ]


100%|██████████| 479/479 [00:02<00:00, 167.61it/s]


Validation scores: [ MAE value:  tensor(0.0934)  ]	[ MSE value:  tensor(0.0160)  ]	[ Pearson Coefficient:  0.42509508942949187  ]
---------------------
Test scores: [ MAE value:  tensor(0.0906)  ]	[ MSE value:  tensor(0.0142)  ]	[ Pearson Coefficient:  0.3827193539517884  ]
---------------------
Train scores: 

  4%|▍         | 18/479 [00:00<00:02, 171.59it/s]

...

Validation scores: [ MAE value:  tensor(0.0648)  ]	[ MSE value:  tensor(0.0071)  ]	[ Pearson Coefficient:  0.7923591454431228  ]
---------------------
Test scores: [ MAE value:  tensor(0.0645)  ]	[ MSE value:  tensor(0.0070)  ]	[ Pearson Coefficient:  0.7565379479701627  ]
---------------------
Train scores: 

  4%|▍         | 18/479 [00:00<00:02, 173.52it/s]

[ MAE value:  tensor(0.0530)  ]	[ MSE value:  tensor(0.0046)  ]	[ Pearson Coefficient:  0.8617601355721737  ]
------------------------------------------------------------------------------------------------------
[Epoch:  25 / 25 ]


100%|██████████| 479/479 [00:02<00:00, 168.53it/s]


Validation scores: [ MAE value:  tensor(0.0649)  ]	[ MSE value:  tensor(0.0070)  ]	[ Pearson Coefficient:  0.7923137728259763  ]
---------------------
Test scores: [ MAE value:  tensor(0.0649)  ]	[ MSE value:  tensor(0.0071)  ]	[ Pearson Coefficient:  0.7568958788050352  ]
---------------------
Train scores: [ MAE value:  tensor(0.0527)  ]	[ MSE value:  tensor(0.0045)  ]	[ Pearson Coefficient:  0.8645342180155735  ]
------------------------------------------------------------------------------------------------------


In [151]:
predf5_train = np.array(fcnnf5.forward(np.concatenate((trainmf, bertword_train,bertsent_train, glovesent_train, gloveword_train),axis=1)).detach().cpu()).reshape(-1)
predf5 = np.array(fcnnf5.forward(np.concatenate((testmf, bertword_test,bertsent_test, glovesent_test, gloveword_test),axis=1)).detach().cpu()).reshape(-1)
predf5_trial = np.array(fcnnf5.forward(np.concatenate((trialmf, bertword_trial,bertsent_trial, glovesent_trial, gloveword_trial),axis=1)).detach().cpu()).reshape(-1)

In [153]:
stats.pearsonr(predf5, testdf['complexity'])

(0.7568958807949003, 3.258471518012556e-171)

## Other ML Models

Gradient-Boosting Regressor

In [136]:
gb1 = GradientBoostingRegressor(random_state=0, n_estimators=100)

featmat_train = np.concatenate((trainmf, gloveword_train, glovesent_train, bertword_train),axis=1)
featmat_test = np.concatenate((testmf, gloveword_test, glovesent_test, bertword_test),axis=1)

labels_train = np.array(traindf['complexity'])
labels_test = np.array(testdf['complexity'])

gb1.fit(featmat_train, labels_train)

predgb1 = gb1.predict(featmat_test)
predgb1_train = gb1.predict(featmat_train)

stats.pearsonr(predgb1, labels_test)

(0.7472669247530377, 1.3948368482915857e-164)

Adaptive-Boosting Regressor

In [137]:
ab1 = AdaBoostRegressor(random_state=0, n_estimators=100)

featmat_train = np.concatenate((trainmf, gloveword_train, glovesent_train, bertword_train),axis=1)
featmat_test = np.concatenate((testmf, gloveword_test, glovesent_test, bertword_test),axis=1)

labels_train = np.array(traindf['complexity'])
labels_test = np.array(testdf['complexity'])

ab1.fit(featmat_train, labels_train)

predab1 = ab1.predict(featmat_test)
predab1_train = ab1.predict(featmat_train)

stats.pearsonr(predab1, labels_test)

(0.7303881735116872, 1.1772726222207615e-153)

Random Forest Regressor

In [139]:
rf1 = RandomForestRegressor(max_depth=7, random_state=0)

featmat_train = np.concatenate((trainmf, gloveword_train, glovesent_train, bertword_train),axis=1)
featmat_test = np.concatenate((testmf, gloveword_test, glovesent_test, bertword_test),axis=1)

labels_train = np.array(traindf['complexity'])
labels_test = np.array(testdf['complexity'])

rf1.fit(featmat_train, labels_train)

predrf1 = rf1.predict(featmat_test)
predrf1_train = rf1.predict(featmat_train)

stats.pearsonr(predrf1, labels_test)

(0.7259772687934656, 6.175532466576186e-151)

# Combine Predictions Ensemble

Using KNN Regressor

In [168]:
neigh = KNeighborsRegressor(n_neighbors=800)

final_train = np.column_stack((predab1_train,predgb1_train,predrf1_train,predf1_train,predf2_train,predf5_train))
final_test = np.column_stack((predab1,predgb1,predrf1,predf1,predf2,predf5))

labels_train = np.array(traindf['complexity'])
labels_test = np.array(testdf['complexity'])

neigh.fit(final_train,labels_train)

pred_final1 = neigh.predict(final_test)

stats.pearsonr(pred_final1, labels_test)

(0.7652538929412512, 3.1624391921968604e-177)

# Submit Final Predictions

In [169]:
submit_df = pd.DataFrame()
submit_df['id'] = testdf['id']
submit_df['predictions'] = pred_final1

submit_df.to_csv('test_res_single.csv', header=False, index=False)